In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lin(x)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor(hidden_channels, 1).to(device)

In [21]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [22]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [23]:
model.reset_parameters()
predictor.reset_parameters()

In [24]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [25]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [26]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.0540
Epoch: 04, Loss: 0.8949
Epoch: 06, Loss: 0.7740
Epoch: 08, Loss: 0.6863
Epoch: 10, Loss: 0.6135
Epoch: 12, Loss: 0.5521
Epoch: 14, Loss: 0.5023
Epoch: 16, Loss: 0.4601
Epoch: 18, Loss: 0.4242
Epoch: 20, Loss: 0.3950
Epoch: 22, Loss: 0.3700
Epoch: 24, Loss: 0.3483
Epoch: 26, Loss: 0.3290
Epoch: 28, Loss: 0.3119
Epoch: 30, Loss: 0.2969
Epoch: 32, Loss: 0.2848
Epoch: 34, Loss: 0.2730
Epoch: 36, Loss: 0.2599
Epoch: 38, Loss: 0.2510
Epoch: 40, Loss: 0.2420
Epoch: 42, Loss: 0.2322
Epoch: 44, Loss: 0.2260
Epoch: 46, Loss: 0.2189
Epoch: 48, Loss: 0.2137
Epoch: 50, Loss: 0.2058
Epoch: 52, Loss: 0.2001
Epoch: 54, Loss: 0.1936
Epoch: 56, Loss: 0.1899
Epoch: 58, Loss: 0.1855
Epoch: 60, Loss: 0.1805
Epoch: 62, Loss: 0.1773
Epoch: 64, Loss: 0.1725
Epoch: 66, Loss: 0.1685
Epoch: 68, Loss: 0.1673
Epoch: 70, Loss: 0.1627
Epoch: 72, Loss: 0.1591
Epoch: 74, Loss: 0.1555
Epoch: 76, Loss: 0.1533
Epoch: 78, Loss: 0.1500
Epoch: 80, Loss: 0.1482
Epoch: 82, Loss: 0.1460
Epoch: 84, Loss:

In [27]:
predictor.lin.weight

Parameter containing:
tensor([[1.0842, 1.0422, 2.1419, 1.0524, 1.1518, 1.7481, 1.0456, 1.2393, 1.3019,
         1.5185, 1.7367, 1.1588, 1.0535, 1.0842, 1.8506, 1.1131, 1.1682, 1.0605,
         1.1757, 1.0859, 1.2841, 1.1821, 1.0134, 0.9961, 1.1382, 0.9820, 1.1549,
         1.1488, 1.0447, 1.1075, 1.1619, 1.3746, 1.3922, 1.0811, 1.1088, 1.0309,
         1.1876, 1.0371, 1.0074, 1.0876, 1.0758, 1.1147, 1.2167, 1.1713, 1.0320,
         1.0565, 1.0112, 1.2501, 1.6069, 0.9239]], device='cuda:0',
       requires_grad=True)

In [28]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
  )
)

In [29]:
emb.weight

Parameter containing:
tensor([[-0.3982, -0.0326,  0.5682,  ..., -0.1477, -0.0900,  0.3533],
        [ 0.0342,  0.0994, -0.1765,  ...,  0.0697,  0.0581,  0.0528],
        [-0.1044, -0.0453,  0.3287,  ..., -0.1218, -0.1817,  0.1411],
        ...,
        [ 0.2918,  0.2104, -0.0511,  ...,  0.1891, -0.1856, -0.1108],
        [-0.1419,  0.1076,  0.0032,  ..., -0.1195,  0.0114,  0.0674],
        [-0.1651, -0.2681, -0.1945,  ...,  0.2112, -0.1108,  0.1758]],
       device='cuda:0', requires_grad=True)

In [30]:
h = model(emb.weight, adj_t)

In [31]:
h

tensor([[-0.3723, -0.7518, -0.4571,  ..., -0.3713,  0.0381,  1.9930],
        [ 0.1383, -0.0612, -0.1413,  ...,  0.0973, -0.1320,  0.1063],
        [-0.0928,  1.0366, -0.2570,  ...,  0.0046, -0.0429,  0.2675],
        ...,
        [ 0.0536, -0.8275,  0.1907,  ..., -0.0541,  0.6051, -0.2835],
        [ 0.2452, -0.5423, -0.3564,  ..., -0.0820,  0.1467,  0.1238],
        [-0.5005,  0.4011, -0.0660,  ..., -0.2405,  0.0297, -0.0952]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [32]:
predictor(h[0], h[411])

tensor([0.9830], device='cuda:0', grad_fn=<SigmoidBackward>)

In [33]:
predictor(h[0], h[2])

tensor([0.0048], device='cuda:0', grad_fn=<SigmoidBackward>)

In [34]:
(h[0] * h[2]).sum()

tensor(-1.1581, device='cuda:0', grad_fn=<SumBackward0>)

In [35]:
(h[0] * h[411]).sum()

tensor(7.4310, device='cuda:0', grad_fn=<SumBackward0>)

In [36]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_33_h.npy', h.cpu().detach().numpy())
torch.save(model, '../../models/graphsage_link_pred/33_model.pt')
torch.save(emb, '../../models/graphsage_link_pred/33_embedding.pt')
torch.save(predictor, '../../models/graphsage_link_pred/33_predictor.pt')